In [ ]:
!pip install seaborn
import torch
import torch.nn as nn # NN modules, nn.Linear, nn.Conv2d, BatchNorm, Loss Functions
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import sklearn
import matplotlib.pyplot as plt

In [ ]:
# LeNet Architecture
# 1x32x32 (Input) -> conv 5x5, s=1, p=0 -> avgPoll s=2, p=0 -> conv 5x5, s=1, p=0 -> avgPoll s=2, p=0
# -> Conv 5x5 to 120 channels -> Linear (84) x Linear (10)

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x)) # num_exemples x 120 x 1 x 1 -> num_exembles x 120
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
# Test
x = torch.randn(64, 1, 32, 32).to('cpu')
model = LeNet().to('cpu')
print(model(x).shape)

In [ ]:
transform = transforms.Compose([
    transforms.Pad(2), # To make images from 28x28 to 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='datasets/', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='datasets/', train=False, download= True, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle= False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = LeNet()
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 0.001)

In [ ]:
def check_acc(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    acc = float(num_correct/float(num_samples))
    print(f'{num_correct}/{num_samples} with accuracy {float(num_correct/float(num_samples))*100:.2f}')
    model.train()
    return acc

In [ ]:
def calculate_classification_metrics(loader, model):
    true = []
    preds = []
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            true.extend(y.cpu())
            preds.extend(predictions.cpu())
    print(sklearn.metrics.classification_report(true, preds, digits=5))
    conf_matrix = confusion_matrix(true, preds)

In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False, )
    checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    for batch_idx, (data, targets) in loop:
        # Send to device
        data = data.to(device)
        targets = targets.to(device)

        # foward
        scores = model(data)
        loss = criterion(scores, targets)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()

        # log
        runningLoss += loss.item()

        # otmizar
        optimizer.step()
        
        # update progress bar
        loop.set_description(f'Epoch [{epoch}/{epochs}]')
        loop.set_postfix(loss = loss.item())

#     save_checkpoint(checkpoint, filename= f'checkpoint_{epoch}.pth.tar')
    trainLoss = runningLoss/ len(train_loader)
    check_acc(train_loader, model)
    check_acc(test_loader, model)

In [ ]:
calculate_classification_metrics(test_loader, model)